# Climate index plots

In [6]:
from ruins.core import build_config
from ruins.core import DataManager
import pandas as pd
import plotly.graph_objects as go


In [2]:
config, dataManager = build_config()

In [8]:
# Make a selection
stati = 'Bremerhaven'
ci_topic = 'ice'
vari = 'Tmax'

In [9]:
INDICES = dict(
    summer='Summer days (Tmax ≥ 25°C)',
    ice='Ice days (Tmax < 0°C)',
    frost='Frost days (Tmin < 0°C)',
    hot='Hot days (Tmax ≥ 30°C)',
    tropic='Tropic nights (Tmin ≥ 20°C)',
    rainy='Rainy days (Precip ≥ 1mm)'
)

In [66]:
def climate_index_agg(ts, index):
     # drop NA
    ts = ts.dropna()

    if index == 'summer':  # summer days
        return (ts >= 25.).groupby(ts.index.year).sum()
    elif index == 'ice':  # ice days
        return (ts < 0.).groupby(ts.index.year).sum()
    elif index == 'frost':  # frost days
        return (ts < 0.).groupby(ts.index.year).sum()
    elif index == 'hot':  # hot days
        return (ts >= 30.).groupby(ts.index.year).sum()
    elif index == 'tropic':  # tropic night
        return (ts >= 20.).groupby(ts.index.year).sum()
    elif index == 'rainy':  # rainy days
        return (ts >= 1.).groupby(ts.index.year).sum()
    else:
        raise ValueError(f"The Index {index} is not supported. Use one of: {','.join(INDICES.keys())}")


def calculate_climate_indices(dataManager: DataManager, station: str, variable: str, ci: str, rolling_windows=(10, 5), rolling_center=True, rcps=('rcp26', 'rcp45', 'rcp85')):
    # load data
    weather = dataManager['weather'].read()[station].sel(vars=variable).to_dataframe()[station].dropna()
    climate = dataManager['cordex_krummh'].read().sel(vars=variable).to_dataframe()
    climate.drop('vars', axis=1, inplace=True)

    # get weather index and rolling
    data = pd.DataFrame(climate_index_agg(weather, ci).astype(int))
    data.columns = [variable]
    data['rolling'] = data.rolling(rolling_windows[0], center=rolling_center).mean()

    # get climate index
    for col in climate.columns:
        df = pd.DataFrame(climate_index_agg(climate[col], ci).astype(int))
        data = pd.merge(data, df, right_index=True, left_index=True, how='outer')

    # get RCP rolling
    for rcp in rcps:
        # select columns that end with rcp
        criteria = [c.endswith(rcp) for c in data.columns]
    
        # subset
        df = data[data.columns[criteria]]
    
        # rolling mean of mean rcp values
        roll = df.mean(axis=1).rolling(rolling_windows[1], center=rolling_center).mean()
        roll = pd.DataFrame(index=roll.index, data={f'{rcp}.rolling': roll.values})
    
        # add back to data
        data = pd.merge(data, roll, right_index=True, left_index=True, how='outer')

    return data



# TEST
data = calculate_climate_indices(dataManager, stati, vari, ci_topic)
#plot_climate_indices(w, c)
data

,Tmax,rolling,CLMcom-ETH.HadGEM2-ES.COSMO-crCLIM-v1-1.r1i1p1.rcp85,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r1i1p1.rcp85,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r2i1p1.rcp85,CLMcom-ETH.M-MPI-ESM-LR.COSMO-crCLIM-v1-1.r3i1p1.rcp85,CLMcom-ETH.NorESM1-M.COSMO-crCLIM-v1-1.r1i1p1.rcp85,CLMcom.CERFACS-CNRM-CM5.CCLM4-8-17.r1i1p1.rcp45,CLMcom.CERFACS-CNRM-CM5.CCLM4-8-17.r1i1p1.rcp85,CLMcom.EC-EARTH.CCLM4-8-17.r12i1p1.rcp26,...,DWD.M-MPI-ESM-LR.EPISODES2018.r1i1p1.rcp85,DWD.M-MPI-ESM-LR.EPISODES2018.r2i1p1.rcp85,DWD.M-MPI-ESM-LR.EPISODES2018.r3i1p1.rcp85,DWD.NorESM1-M.EPISODES2018.r1i1p1.historical,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp26,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp45,DWD.NorESM1-M.EPISODES2018.r1i1p1.rcp85,rcp26.rolling,rcp45.rolling,rcp85.rolling
time,,,,,,,,,,,,,,,,,,,,,
1949,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1950,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1952,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,NaN,NaN,0.0,11.0,0.0,0.0,0.0,0.0,0.0,3.0,...,2.0,6.0,0.0,NaN,13.0,1.0,1.0,6.381976,3.415070,1.359624
2093,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,...,0.0,0.0,0.0,NaN,5.0,2.0,3.0,6.505882,3.911111,1.260094
2094,NaN,NaN,0.0,0.0,0.0,4.0,2.0,2.0,0.0,10.0,...,0.0,0.0,4.0,NaN,12.0,0.0,3.0,6.023529,4.613228,1.185760


In [113]:
def plot_climate_indices(data: pd.DataFrame, rcps=('rcp26', 'rcp45', 'rcp85')):
    fig = go.Figure()
    
    # add the weather indices
    fig.add_trace(
        go.Scatter(x=data.index, y=data.iloc[:, 0].values, mode='markers', marker=dict(color='steelblue', size=5), name='Weather', hovertemplate="%{y} days in %{x}<extra></extra>")
    )

    # add the rolling mean
    fig.add_trace(
        go.Scatter(x=data.index, y=data['rolling'], mode='lines', line=dict(color='steelblue', width=3), name='Rolling mean (10 years)', hovertemplate="%{y:.1f} days in %{x}<extra></extra>")
    )

    for i, rcp in enumerate(rcps):        
        # melt down to only this rcp
        df = data.melt(value_vars=data.columns[[c.endswith(rcp) for c in data.columns]], ignore_index=False)
        fig.add_trace(
            go.Scattergl(x=df.index, y=df.value.values, mode='markers', marker=dict(color=f'rgba(127, 127, 127, {0.2 + 0.1 * i / 3})', size=5), name=rcp.upper(), meta=[rcp.upper()], hovertemplate="%{y} days in %{x}<extra>%{meta[0]}</extra>")
        )

        # add the rolling mean
        roll = data[[f'{rcp}.rolling']]
        fig.add_trace(
            go.Scatter(x=roll.index, y=roll.values.flatten(), mode='lines', line=dict(width=5), name=f'Rolling mean of {rcp.upper()}', meta=[rcp.upper()], hovertemplate="%{y:.1f} days in %{x}<extra>%{meta[0]}</extra>")
        )


    fig.update_layout(
        template='plotly_white',
        legend=dict(orientation='h'),
        yaxis=dict(title='Number of days'),
    )
    return fig

plot_climate_indices(data)


In [105]:
data[[f'{rcp}.rolling']].flatten()


AttributeError: 'DataFrame' object has no attribute 'flatten'